- title: The Statefulness of Systems and Software
- date: 2020-07-25
- modified: 2020-07-25 00:00
- category: Engineering
- tags: Systems Engineering, Software Engineering
- slug: StatefulSystems
- summary: Breakdown of how most systems have state
- header_cover: images/systems-engineering2.jpg

Most of the time in software engineering, designers tend to shun or gravitate towards utilizing the internal state of the module they're developing. This post isn't to discuss that. Rather, it's to show that the concept of *state* is a embedded in a wide range of structures. But first, let's try to define what's meant by state itself.

# The Stateful Nature of Things
Usually in signal processing and many other software applications, we like to think of things as a function. Some set of inputs **x** is passed to a function $f()$, and we get a response **y** from that function.

![Time Invariant System](../images/TI_System.png)
<figcaption><i>Fig. 1 - Time-Invariant System Flow</i></figcaption>

A lot of things seem to work that way: a light bulb filament gives off light proportionate to the energy applied to it. This lack of statefulness implies *time invariance*: It doesn't matter when energy is applied, the filament gives off light.

It's a convenient way to understand systems. This is how *data driven* systems are designed: Data goes in, transformed data comes out. But time invariance places specific requirements on the inputs themselves. When such a system is given an *impulse*, it changes its response only then and provides a transient response afterwards (which can include no response at all!). Take for example listening to the radio: if the signal is disrupted, but what if we don't want the music to abruptly stop? Many streaming services have *buffering* built into them to cache the stream so that disruptions in the signal are masked by the buffer itself. In a sense, this buffer cache is providing a state to the application: as long as it isn't empty, normal operations can progress!

Digital control systems are usually designed considering that the input signal may be *latent* or *dropped* an update, but they still have requirements to produce their output following some periodic. Because of this, a lot of signal processing implementations follow this model.

![Time Invariant System](../images/nonTI_System.png)
<figcaption><i>Fig. 2 - Data-Robust System Flow</i></figcaption>

The above model now has two **triggers** for generating an output: one is upon receipt of new data, and another is when the timer expires. This simple model guarantees a data rate of at-least the timer rate, which is applicable in a lot of design circumstances. But the internal **state** of the controller changes based on the trigger because the logic for when no new data is available will activate or deactivate based on the trigger.

# State Modeling
Lets take a look at some sample code describing a system like in Figure 2.

```cpp
/*
 * Listing 1 - Example of a stateful program
 */

#include "MyInputApi.h"
#include "MyOutputApi.h"
#include "MyOverseerApi.h"
#include "MyTransformApi.h"

#include <iostream>
#include <cstdlib>

int main(int, char** argv)
{
  MyTransformApi hiddenFunc;
  srand(atoi(argv[1]));
  try
  {
    // Initialize the interfaces
    MyInputApi inputIf;
    inputIf.connect();

    MyOutputApi outputIf;
    outputIf.connect();

    std::string buf;
    while(MyOverseerApi::getInstance().keepRunning())
    {
      // buf.clear();
      MyTransformApi::OutData output;
      auto status = inputIf.getNext(buf);

      if(status)
      {
        // Got new data
        output = hiddenFunc.magic(buf);
      }
      else
      {
        // Data wasn't available
        output = hiddenFunc.extrapolateFromLast();
      }
      outputIf.send(output);
    }
  }
  catch(...)
  {
    std::cout << "Caught an exception, so aborting!" << std::endl;
  }
  return 0;
}
```

Here's a basic example of it running (pardon the bit of Python to invoke it).

In [1]:
# Built before running the notebook...
!../code/SwSys_StateModeling/StateModelingExample 3

I got valid data! [1001] --- I got valid data! [1002] --- I got valid data! [1003] --- I got valid data! [1004] --- I got valid data! [1005] --- I got valid data! [1006] --- I got valid data! [1007] --- I got valid data! [1008] --- I got valid data! [1009] --- I got valid data! [1010] --- I got valid data! [1011] --- I got valid data! [1012] --- I got valid data! [1013] --- I got valid data! [1014] --- I got valid data! [1015] --- I got valid data! [1016] --- I got valid data! [1017] --- I got valid data! [1018] --- I got valid data! [1019] --- Extrapolating from past data [2020] --- I got valid data! [2021] --- I got valid data! [2022] --- I got valid data! [2023] --- I got valid data! [2024] --- I got valid data! [2025] --- I got valid data! [2026] --- I got valid data! [2027] --- I got valid data! [2028] --- I got valid data! [2029] --- I got valid data! [2030] --- I got valid data! [2031] --- I got valid data! [2032] --- I got valid data! [2033] --- I got valid data! [2034] --- I g

In [2]:
!../code/SwSys_StateModeling/StateModelingExample 5

I got valid data! [1001] --- Extrapolating from past data [2002] --- I got valid data! [2003] --- Extrapolating from past data [3004] --- I got valid data! [3005] --- I got valid data! [3006] --- Extrapolating from past data [4007] --- Extrapolating from past data [5008] --- Extrapolating from past data [6009] --- I got valid data! [6010] --- I got valid data! [6011] --- I got valid data! [6012] --- I got valid data! [6013] --- I got valid data! [6014] --- I got valid data! [6015] --- I got valid data! [6016] --- I got valid data! [6017] --- I got valid data! [6018] --- Extrapolating from past data [7019] --- Extrapolating from past data [8020] --- I got valid data! [8021] --- I got valid data! [8022] --- I got valid data! [8023] --- I got valid data! [8024] --- I got valid data! [8025] --- I got valid data! [8026] --- I got valid data! [8027] --- I got valid data! [8028] --- I got valid data! [8029] --- I got valid data! [8030] --- I got valid data! [8031] --- I got valid data! [8032]

In general, the above code is very functional and has a simple decision flow that can be described like this:

![Decision Flow Diagram of Example Code](../images/SysSw_StateModeling_DecisionModel.png)

<figcaption><i>Fig 3. Decision Flow Diagram of Example Code</i></figcaption>

The code itself has a very simple flow: open the interfaces, main-loop process with backup processing when data is unavailable, and close the interfaces. And if anything goes wrong, the process handles exceptions before shutting down.

What's interesting is that each major block is a unique phase of operation. It takes some amount of time to execute, and *transition* to the next operation depends on how the previous concluded. In essence, we can show that this program has different *states of operation* at any given time. Thus, we can also describe this process with a **state model**:

![State Model of the Example Code](../images/SysSw_StateModeling_StateModel.png)
<figcaption><i>Fig 4. State Modeling of the Example Code</i></figcaption>

It's been said hundreds of different ways, but this resonates in many cases:

> It should be understood that the model is, at best, a surrogate for the actual system, whose precision is subject to the assumptions and approximations made by the control designer.

*- Optimal Control & Estimation*

And that's just it, what we see above is merely a model describing the internal state of the process. It reveals a different facet, a different interpretation of the design which can highlight behaviors not considered by other models (in this case, the Decision Flow model)

# State Machines vs State Models
Now that we can see that software can be described as having a *runtime state*, what then is the difference between this and a machine? Concisely, a **state machine** is a construct which *senses* events/triggers, then *follows* a pre-defined set of rules governing transition, and finally *executes* a specific set of system behaviors which may provide an additional trigger. This is a nuanced definition, and we'll explore the differences between a stateful system versus a state machine.

The key differentiation here is that something has to reconcile **ALL** events/triggers. In design, it isn't sufficient if some parts can detect a trigger (like the wrapping try-catch in the listing above). For example, in *Listing 1*, immediately after calling `connect()` with each of the interfaces, the program immediately transitions into the execution loop without any evaluation of events.

Another differentiator is that **state machines are a software design pattern**! *Listing 1* is purposefully implemented naively to highlight the anti-pattern. To maintain flexibility, the insertion or removal of a state and its associated triggers **should not** require a redesign of the underlying code. This design pattern isn't unique (it often is implemented via other equally flexible design patterns), but let's explore one of these.

# The State Machine Pattern

There are a lot of blog posts and Stack Overflow answers which describe how to implement a robust state machine following Functional or Object Oriented Programming paradigms. For brevity, let's review an example implementation in OOP\[[1](Ref1)\]:

![State Machine via Chain of Responsibility](../images/CoR_SM_Diagram.png)
<figcaption><i>Fig 5. State Machine Design Pattern via Chain of Responsibility\[[2](Ref2)\]</i></figcaption>

```cpp
// TODO
```

## References

<a name="Ref1">1.</a> Lafreniere, David. "State Machine Design in C++". Code Project, 2019. [Online] Available: https://www.codeproject.com/Articles/1087619/State-Machine-Design-in-Cplusplus-2

<a name="Ref2">2.</a> "Chain of Responsibility". Open Source Matters, 2006. [Online] Available: https://www.oodesign.com/chain-of-responsibility-pattern.html